# Physionet 2017 | ECG Rhythm Classification
## 4. Train Model (CPU Test)
### Sebastian D. Goodfellow, Ph.D.

<br>
# Setup Noteboook

In [1]:
# Import 3rd party libraries
import os
import sys
import numpy as np
import pickle

# Deep learning libraries
import tensorflow as tf

# Import local Libraries
sys.path.insert(0, r'C:\Users\sebig\Documents\code\deep_ecg')
from utils.plotting.time_series import plot_time_series_widget
from utils.data.labels.one_hot_encoding import one_hot_encoding
from utils.devices.device_check import print_device_counts, get_device_names
from train.train import train
from model.model import Model

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# 1. Load ECG Dataset

In [2]:
# Set path
path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'training')

# Set sample rate
fs = 300

# Unpickle
with open(os.path.join(path, 'training_60s.pickle'), "rb") as input_file:
    data = pickle.load(input_file)

In [3]:
# Get training data
x_train = data['data_train'].values.reshape(data['data_train'].shape[0], data['data_train'].shape[1], 1)
y_train = data['labels_train']['label_int'].values.reshape(data['labels_train'].shape[0], 1).astype(int)

# Get validation data
x_val = data['data_val'].values.reshape(data['data_val'].shape[0], data['data_val'].shape[1], 1)
y_val = data['labels_val']['label_int'].values.reshape(data['labels_val'].shape[0], 1).astype(int)

# Print dimensions
print('x_train dimensions: ' + str(x_train.shape))
print('y_train dimensions: ' + str(y_train.shape))
print('x_val dimensions: ' + str(x_val.shape))
print('y_val dimensions: ' + str(y_val.shape))

x_train dimensions: (5774, 18000, 1)
y_train dimensions: (5774, 1)
x_val dimensions: (2475, 18000, 1)
y_val dimensions: (2475, 1)


In [4]:
# One hot encoding array dimensions
y_train_1hot = one_hot_encoding(labels=y_train.ravel(), classes=len(np.unique(y_train.ravel())))
y_val_1hot = one_hot_encoding(labels=y_val.ravel(), classes=len(np.unique(y_val.ravel())))

# Print dimensions
print('x_train dimensions: ' + str(x_train.shape))
print('y_train dimensions: ' + str(y_train.shape))
print('y_train_1hot dimensions: ' + str(y_train_1hot.shape))
print('x_val dimensions: ' + str(x_val.shape))
print('y_val dimensions: ' + str(y_val.shape))
print('y_val_1hot dimensions: ' + str(y_val_1hot.shape))

x_train dimensions: (5774, 18000, 1)
y_train dimensions: (5774, 1)
y_train_1hot dimensions: (5774, 3)
x_val dimensions: (2475, 18000, 1)
y_val dimensions: (2475, 1)
y_val_1hot dimensions: (2475, 3)


In [5]:
# Label lookup
label_lookup = {'N': 0, 'A': 1, 'O': 2, '~': 3}

# Label dimensions
print('Train: Classes: ' + str(np.unique(y_train.ravel())))
print('Train: Count: ' + str(np.bincount(y_train.ravel())))
print('Val: Classes: ' + str(np.unique(y_val.ravel())))
print('Val: Count: ' + str(np.bincount(y_val.ravel())))

Train: Classes: [0 1 2]
Train: Count: [3553  531 1690]
Val: Classes: [0 1 2]
Val: Count: [1523  227  725]


In [6]:
# Label dictionary
label_list = ['Normal Sinus Rhythm', 'Atrial Fibrillation', 'Other Rhythm']

# PLot times series
plot_time_series_widget(time_series=x_train, labels=y_train, fs=fs, label_list=label_list)

interactive(children=(IntSlider(value=2886, description='index', max=5773), Output()), _dom_classes=('widget-i…

# 2. Device Check

In [7]:
# Get GPU count
print_device_counts()

Workstation has 1 CPUs.
Workstation has 0 GPUs.


# 3. Initialize Model

In [8]:
# Set save path for graphs, summaries, and checkpoints
save_path = r'C:\Users\sebig\Desktop\tensorboard\deep_ecg\test'

# Set model name
model_name = 'test_10'

# Maximum number of checkpoints to keep
max_to_keep = 20

# Set randome states
seed = 0                                    
tf.set_random_seed(seed)                      

# Get training dataset dimensions
(m, length, channels) = x_train.shape  

# Get number of label classes
classes = y_train_1hot.shape[1]                  

# Choose network
network_name = 'DeepECG'

# Set network inputs
network_parameters = dict(
    length=length,
    channels=channels, 
    classes=classes, 
    seed=seed,  
)

# Create model
model = Model(
    model_name=model_name, 
    network_name=network_name, 
    network_parameters=network_parameters, 
    save_path=save_path,
    max_to_keep=max_to_keep
)

# 7. Train Model

In [9]:
# Set hyper-parameters
epochs = 10
minibatch_size =  1
learning_rate = 0.001            

# Train model
train(model=model, x_train=x_train[0:1], y_train=y_train_1hot[0:1], x_val=x_val[0:1], y_val=y_val_1hot[0:1],
      learning_rate=learning_rate, epochs=epochs, mini_batch_size=minibatch_size)